# **Dynamical system**
by: Edwin Saavedra C.

In [1]:
#%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wd
import sympy as sp
from IPython.display import display

In [2]:
## MATPLOTLIB rcParams
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.labelweight'] = 'medium'
plt.rcParams['axes.labelpad'] = 5.0
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['axes.edgecolor'] = 'gray'

In [3]:
def buildEquations(pi):
    x,c = sp.symbols('x c')
    dxdtEq = sp.Eq(pi[4]*x*c/(pi[1]+c)*(1-x) - pi[5]*x,0)
    dcdtEq = sp.Eq(-pi[0]*x*c/(pi[1]+c)*(1-x) + pi[2]*sp.exp(-pi[3]*x)*(1-c),0)
    return dxdtEq,dcdtEq

In [4]:
def stream(x,c,pi):
    dxdt =  pi[4]*x*c/(pi[1]+c)*(1-x) - pi[5]*x
    dcdt = -pi[0]*x*c/(pi[1]+c)*(1-x) + pi[2]*np.exp(-pi[3]*x)*(1-c)
    return dxdt,dcdt

def rateAsColors(x,c,pi):
    rateC = pi[0]*c/(pi[1]+c)*(1-x)  #Substrate consumption
    rateQ = pi[2]*np.exp(-pi[3]*x)   #Water inflow
    rate0 = np.zeros_like(rateC)
    return np.stack(([rateC,rate0,rateQ]),axis=-1)

xlin = np.linspace(0,1.0,31)
clin = np.linspace(0,1.0,31)
x,c   = np.meshgrid(xlin,clin)

In [5]:
piBoxVals = [1,0.1,0.7,1.5,0.7,0.1]
piBoxes   = [wd.HTMLMath(str(i)) for i in range(6)]

paramValues = [2.0,2.0,0.20,1.4,1.5,0.7,0.2,2.0]
parameters = [r'X_{\rm max}',r'C_\infty',r'K_C',r'Q',r'\beta',r'Y',r'k_{\rm die}',r'\hat{\mu}']
KSliders   = [wd.FloatSlider(value=v,min=0,max=2,step=0.01,description=f'${k}$') for k,v in zip(parameters,paramValues)]

In [6]:
def buildPiBoxes():
    global piBoxVals
    global piBoxes
    for i,(p,val) in enumerate(zip(piBoxes,piBoxVals)):
        p.value = rf"$\pi_{{{i}}} = {{{val}}}$"

def updatePi(change):
    global piBoxVals
    piBoxVals[0] = KSliders[0].value/KSliders[1].value
    piBoxVals[1] = KSliders[2].value/KSliders[1].value
    piBoxVals[2] = KSliders[3].value/KSliders[7].value
    piBoxVals[3] = KSliders[4].value
    piBoxVals[4] = KSliders[5].value
    piBoxVals[5] = KSliders[6].value/KSliders[7].value
    
    piBoxVals = [round(p,2) for p in piBoxVals]
    buildPiBoxes()
    return None
    
for slider in KSliders:
    slider.observe(updatePi,'value')
    
updatePi(1)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/edsaac/dashboards/HEAD?urlpath=voila%2Frender%2FExponentialGrowth.ipynb)

In [7]:
whitespace = wd.HTML("&nbsp;&nbsp;<b>&#8594;</b>&nbsp;&nbsp;")
layout = wd.Layout(justify_content='center',align_items='center')

eqTex0 = wd.HTMLMath(r'''\begin{equation}
\begin{array}{rcl}
    \dfrac{dC}{dt} &=& -\hat{\mu}X\dfrac{C}{K_C + C}\left(1-\dfrac{X}{X_{\rm max}}\right) + Q(C_\infty-C)\exp{\left(-\beta \dfrac{X}{X_{\rm max}}\right)} \\
    \dfrac{dX}{dt} &=& Y\hat{\mu}X\dfrac{C}{K_C + C}\left(1-\dfrac{X}{X_{\rm max}}\right) - k_{\rm die}X
\end{array}
\end{equation}''',layout=layout)

eqTex1_1 = wd.HTMLMath(r'''\begin{equation*}
\begin{array}{rcl}
    c &=& C/C_\infty \\
    x &=& X/X_{\rm max} \\
    \tau &=& \hat{\mu}t \\
\end{array}
\end{equation*}''')

eqTex1_2 = wd.HTMLMath(r'''\begin{equation*}
\begin{array}{rcl}
    \pi_0 &=& X_{\rm max}/C_\infty \\
    \pi_1 &=& K_C/C_\infty \\
    \pi_2 &=& Q/\hat{\mu} \\
    \pi_3 &=& \beta \\
    \pi_4 &=& Y \\
    \pi_5 &=& k_{\rm die}/\hat{\mu}
\end{array}
\end{equation*}''',)

eqTex1 = wd.HBox([eqTex1_1,whitespace,eqTex1_2],layout=layout)

eqTex2 = wd.HTMLMath(r'''\begin{equation}
\begin{array}{rcl}
    \dfrac{dc}{d\tau} &=& -\pi_0 x\dfrac{c}{\pi_1 + c}(1-x) + \pi_2(1-c)\exp{\left(-\pi_3 x\right)} \\
    \dfrac{dx}{d\tau} &=& \pi_4 x\dfrac{c}{\pi_1 + c}(1-x) - \pi_5 x
\end{array}
\end{equation}''',layout=layout)

eqTexs = [eqTex0,eqTex1,eqTex2]

In [8]:
kw_streams = {'density':1,'linewidth':1.5,'arrowsize':0.8,
              'arrowstyle':'->','color':[1,1,1,0.5],'minlength':0.2}

output = wd.Output(layout=layout)

@output.capture(clear_output=True)
def plotStream(x,c,dx,dc,imBands):
    eq1,eq2 = buildEquations(piBoxVals)
    ans = sp.solvers.solvers.nsolve((eq1,eq2),sp.symbols('x c'),(0.8,0.4))
    
    fig,ax = plt.subplots(figsize=[8,6])
    ax.imshow(imBands,extent=[0,1,0,1],zorder=1)
    ax.streamplot(x,c,dx,dc,**kw_streams,zorder=2)
    
    ## Steady states
    ax.scatter([0],[1],s=200,c='limegreen',label='Trivial\nsteady-state',zorder=7,clip_on=False,ec='k')
    ax.scatter([float(ans[0])],[float(ans[1])],s=200,c='darkorange',label='Non-trivial\nsteady-state',zorder=7,ec='k')
    
    ax.grid(True,ls='dashed',lw=1,c=[0.5,0.5,0.5,0.5])
    ax.set_ylabel("$c$ [-]",)
    ax.set_xlabel("$x$ [-]")
    ax.legend(loc='upper right')
    ax.set(xlim=[0,1],ylim=[0,1])
    
    plt.subplots_adjust(left=0.1, right=0.65, top=0.85)
    cax = fig.add_axes([0.7,0.45,0.2,0.2])
    
    Legend = np.dstack((x, np.zeros_like(x), c))
    cax.imshow(Legend, origin="lower", extent=[0,1,0,1])
    cax.set_ylabel("Flow rate",)
    cax.set_xlabel("Reac rate")
    cax.set_xticks([0,1])
    cax.set_yticks([0,1])
    
    plt.show();

def updateFig(b):
    dx,dc = stream(x,c,piBoxVals)
    imBands = rateAsColors(x,c,piBoxVals)
    plotStream(x,c,dx,dc,imBands);
    return None

updateFig(1);

plotButton = wd.Button(description="Plot!",icon='drafting-compass')
plotButton.on_click(updateFig)

gs = wd.GridspecLayout(12,2)
gs[0:5,0] = wd.VBox(KSliders)
gs[0:4,1] = wd.VBox(piBoxes)
gs[4,1] = plotButton
gs[5:,:] = wd.HBox([output])

accordion = wd.Accordion(eqTexs,layout=wd.Layout(align_items='stretch'))
accordion.selected_index = None
accordion.set_title(0,'Governing ODEs')
accordion.set_title(1,'Nondimensionalization')
accordion.set_title(2,'Non-dimensional ODEs')

mainBox = wd.VBox([accordion,gs])
display(mainBox)